# **Market Basket Analysis**

**1. Giriş**

<div>
<img src="https://cdn.shortpixel.ai/client/to_avif,q_glossy,ret_img,w_640/https://labs.sogeti.com/wp-content/uploads/2019/01/Shopping.jpg" width="400"/>
</div>

Pazarlamacılar genellikle alışveriş sepetlerinde çok sık ve bir arada bulunan anahtar ürün çiftlerini bulmakla görevlendirilir. Bu notebook, bunun Apriori algoritması kullanılarak nasıl elde edilebileceğini açıklar. Bu ürün çiftlerini, bir müşteri daha sepete eklemişse, başka bir ürün önermek için temel oluşturur.

Ürün çifti kombinasyonlarının bazı popüler örnekleri ekmek ve reçel kadar önemsiz veya mantıklı olabilirken, bira ve çocuk bezi gibi daha şaşırtıcı olabilir.

Teknik olarak, bir çiftteki ürünlerin sırası da önemlidir. Bir çiftte her zaman baskın bir ürün vardır, örneğin ekmek satışlarının daha fazla reçel satışına yol açıp açmadığı veya tam tersi, her biri farklı puan alan iki farklı durumdur.

**1.1 Problem Tanımı**

Bu çalışmada, etkili ürün önerileri yapabilmek için özel bir algoritma (Apriori algoritması) kullanarak hangi ürünleri set halinde satılabileceği ve önerilerde bulunulabileceği gösterilecektir.


## 1.2 Modülleri İçe Aktar

In [38]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 2. Birliktelik Analizi  

Apriori Algoritmasının ismi, bilgileri bir önceki adımdan elde ettiği için “prior (önceki)” anlamında Apriori’dir. Bu algoritma temelinde iteratif (tekrarlayan) bir niteliğe sahiptir ve hareket bilgileri içeren veri tabanlarında sık geçen öğe kümelerinin keşfedilmesinde kullanılır.
Birliktelik kuralı madenciliği, tüm sık geçen öğelerin bulunması ve sık geçen bu öğelerden güçlü birliktelik kurallarının üretilmesi olmak üzere iki aşamalıdır. Birliktelik kuralının ilk aşaması için kullanılan Apriori Algoritması, sık geçen öğeler madenciliğinde kullanılan en popüler ve klasik algoritmadır. Bu algoritmada özellikler ve veri, “boolean” ilişki kuralları ile değerlendirilir.

Confidence değeri 0 ile 1 arasındadır. Yüksek değer kuralımızın çok güvenli(doğru) olduğunu belirtir.

<div>
<img src="https://miro.medium.com/max/659/1*wYdSft4v1_9fGj7SFC9zeg.png" width="400"/>
</div>

Lift değeri 0 ile ∞(sonsuz) arasındadır. Yüksek değer, kuralımızın ilginç olduğunu gösterir.

<div>
<img src="https://miro.medium.com/max/680/1*sGp9xGMUQhcDraH-VJfchg.png" width="400"/>
</div>

<div>
<img src="https://miro.medium.com/max/609/1*sZgbk3l9t6-SDCHJnduE6w.png" width="400"/>
</div>


<div>
<img src="https://miro.medium.com/max/716/1*arKkfXPBZUQygAurKp6NPg.png" width="400"/>
</div>


<div>
<img src="https://miro.medium.com/max/473/1*AWuq4moifmyKmQ4jCFp79g.png" width="400"/>
</div>


In [39]:
data = pd.read_csv("/content/basket_analysis_dataset.csv")
data

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,NaN,NaN,NaN,NaN
311,Meat,Milk,Pencil,NaN,NaN,NaN,NaN
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,NaN,NaN,NaN,NaN,NaN


In [40]:
column_values = data[[str(i) for i in range(data.shape[-1])]].values.ravel()
unique_values =  pd.unique(column_values)
unique_values = unique_values[~pd.isnull(unique_values)]
unique_values

array(['Bread', 'Wine', 'Eggs', 'Meat', 'Cheese', 'Pencil', 'Diaper',
       'Milk', 'Bagel'], dtype=object)

In [41]:
transaction_data = []
for i in range(data.shape[0]):
    transaction_data.append([str(data.values[i,j]) for j in range(data.shape[-1])])

transaction_data = np.array(transaction_data)
te = TransactionEncoder()
transaction_data = te.fit_transform(transaction_data)
transaction_data = transaction_data.astype("int")
transaction_data = pd.DataFrame(transaction_data, columns = te.columns_)
transaction_data.drop('nan', inplace=True, axis=1)
transaction_data

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...
310,0,1,1,0,1,0,0,0,0
311,0,0,0,0,0,1,1,1,0
312,0,1,1,1,1,1,0,1,1
313,0,0,1,0,0,1,0,0,0


In [42]:
frequent_itemsets = apriori(transaction_data, min_support = 0.15, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bagel, Bread)"


In [43]:
print("Kural Sayısı:", len(frequent_itemsets))

Kural Sayısı: 54


In [44]:
rules1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.30)
rules1 = rules1.sort_values(['confidence'], ascending=False)
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
122,"(Eggs, Milk, Meat)",(Cheese),0.177778,0.501587,0.152381,0.857143,1.708861,0.063210,3.488889
98,"(Milk, Meat)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137
79,"(Eggs, Meat)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667
86,"(Milk, Eggs)",(Cheese),0.244444,0.501587,0.196825,0.805195,1.605293,0.074215,2.558519
123,"(Cheese, Milk, Eggs)",(Meat),0.196825,0.476190,0.152381,0.774194,1.625806,0.058655,2.319728
...,...,...,...,...,...,...,...,...,...
112,(Cheese),"(Wine, Milk)",0.501587,0.219048,0.161905,0.322785,1.473583,0.052033,1.153182
130,(Meat),"(Cheese, Milk, Eggs)",0.476190,0.196825,0.152381,0.320000,1.625806,0.058655,1.181139
47,(Milk),(Diaper),0.501587,0.406349,0.155556,0.310127,0.763202,-0.048264,0.860521
131,(Cheese),"(Eggs, Milk, Meat)",0.501587,0.177778,0.152381,0.303797,1.708861,0.063210,1.181010


In [45]:
print("Oluşan Kural Sayısı:", len(rules1))

Oluşan Kural Sayısı: 134


# **Referanslar**

*   [Association Analysis](https://www.kaggle.com/xvivancos/market-basket-analysis)
*   [Market Basket Analysis](https://www.kaggle.com/roshansharma/market-basket-analysis)
*   [Birliktelik Analizi](https://www.veribilimiokulu.com/python-ile-birliktelik-kurallari-analizi-association-rules-analysis-with-python/)
*   [Appriori Algoritması](https://medium.com/@cotur/apriori-algoritmas%C4%B1-a90a38856373)